# Predictive Trading with Time-Aware Neural Networks
Testing irregular message frequencies and temporal memory effects

In [1]:
# Restart kernel to reload fixed code
import importlib
import sys
import os
from dotenv import load_dotenv

load_dotenv()
ROOT = os.getenv('ROOT')
sys.path.append(ROOT)

# Clear any cached imports
if 'src.neural.neuron' in sys.modules:
    del sys.modules['src.neural.neuron']
if 'src.evolution.individual' in sys.modules:
    del sys.modules['src.evolution.individual']

import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from src.evolution import Individual

## Generate Predictive Price Data
Sine wave oscillations with trading noise between 900-1100

In [2]:
n_points = 5000
base_times = np.arange(n_points)
prices = 1000 + 100 * np.sin(2 * np.pi * base_times / 1000) + np.random.normal(0, 5, n_points)
prices = np.clip(prices, 900, 1100)

## Create Irregular Timestamps
Simulate websocket rush periods with variable message frequency

In [3]:
timestamps = []
price_data = []
current_time = 0.0

for i in range(n_points):
    rush_period = (i % 500) < 50  # Rush every 500 points for 50 points
    time_increment = 0.1 if rush_period else 1.0  # Fast vs normal frequency
    
    current_time += time_increment
    timestamps.append(current_time)
    price_data.append(prices[i])

timestamps = np.array(timestamps)
price_data = np.array(price_data)

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate time differences
time_diffs = np.diff(timestamps[:1000])

# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add price trace on primary y-axis
fig.add_trace(
    go.Scatter(x=timestamps[:1000], y=price_data[:1000], name="Price", line=dict(color="blue")),
    secondary_y=False,
)

# Add time differences on secondary y-axis
fig.add_trace(
    go.Scatter(x=timestamps[:999], y=time_diffs, name="Time Diff", line=dict(color="red")),
    secondary_y=True,
)

# Set y-axes titles
fig.update_yaxes(title_text="Price", secondary_y=False)
fig.update_yaxes(title_text="Time Between Messages", secondary_y=True)

# Set layout
fig.update_layout(
    title_text="Price Movements with Rush Periods (Dual Y-Axis)",
    width=1200,
    height=600
)

fig.show()

## Time-Aware Trading Simulation
Neural network handles irregular timing with temporal memory

In [10]:
def simulate_trading(individual, prices, timestamps):
    individual.reset_state()
    portfolio_value = 1.0
    position = 0  # 0=no position, 1=long
    entry_price = 0.0
    
    actions = []
    executed_actions = []  # Track what actions were actually executed
    values = []
    
    for i, (price, timestamp) in enumerate(zip(prices, timestamps)):
        normalized_price = (price - 1000) / 100
        raw_action = individual.get_action(normalized_price, timestamp)
        
        # Enforce position constraints
        if position == 0:  # No position
            if raw_action == 2:  # Buy signal
                position = 1
                entry_price = price
                executed_action = 2  # Buy executed
            elif raw_action == 0:  # Sell signal when no position
                executed_action = 1  # Convert to hold (can't sell when not long)
            else:
                executed_action = 1  # Hold
        else:  # Long position
            if raw_action == 0:  # Sell signal
                position = 0
                portfolio_value *= price / entry_price
                executed_action = 0  # Sell executed
            elif raw_action == 2:  # Buy signal when already long
                executed_action = 1  # Convert to hold (can't buy when already long)
            else:
                executed_action = 1  # Hold
        
        actions.append(raw_action)  # Original network output
        executed_actions.append(executed_action)  # What actually happened
        values.append(portfolio_value)
    
    return np.array(actions), np.array(executed_actions), np.array(values)

In [14]:
individual = Individual()
raw_actions, executed_actions, portfolio_values = simulate_trading(individual, price_data, timestamps)

print(f"Final portfolio value: {portfolio_values[-1]:.4f}")
print(f"Raw network actions: Hold={np.sum(raw_actions==1)}, Sell={np.sum(raw_actions==0)}, Buy={np.sum(raw_actions==2)}")
print(f"Executed actions: Hold={np.sum(executed_actions==1)}, Sell={np.sum(executed_actions==0)}, Buy={np.sum(executed_actions==2)}")

# Count invalid actions
invalid_sells = np.sum((raw_actions == 0) & (executed_actions == 1))  # Tried to sell when no position
invalid_buys = np.sum((raw_actions == 2) & (executed_actions == 1))   # Tried to buy when already long

print(f"Invalid actions: {invalid_sells} sells when no position, {invalid_buys} buys when already long")

Final portfolio value: 1.0856
Raw network actions: Hold=4770, Sell=138, Buy=92
Executed actions: Hold=4991, Sell=4, Buy=5
Invalid actions: 134 sells when no position, 87 buys when already long


In [15]:
# Create multi-panel plotly visualization using executed actions
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Trading Actions on Price Chart (Executed Actions)', 'Portfolio Value Over Time', 'Message Frequency (Rush Periods Visible)'),
    vertical_spacing=0.08
)

# Panel 1: Price with buy/sell signals (use executed_actions)
fig.add_trace(
    go.Scatter(x=timestamps[:2000], y=price_data[:2000], name="Price", line=dict(color="blue", width=1)),
    row=1, col=1
)

# Buy signals (executed)
buy_mask = executed_actions[:2000] == 2
buy_signals = timestamps[:2000][buy_mask]
buy_prices = price_data[:2000][buy_mask]
fig.add_trace(
    go.Scatter(x=buy_signals, y=buy_prices, mode='markers', name="Buy (Executed)", 
               marker=dict(color="green", symbol="triangle-up", size=8)),
    row=1, col=1
)

# Sell signals (executed)
sell_mask = executed_actions[:2000] == 0
sell_signals = timestamps[:2000][sell_mask]
sell_prices = price_data[:2000][sell_mask]
fig.add_trace(
    go.Scatter(x=sell_signals, y=sell_prices, mode='markers', name="Sell (Executed)",
               marker=dict(color="red", symbol="triangle-down", size=8)),
    row=1, col=1
)

# Invalid actions (converted to hold)
invalid_mask = (raw_actions[:2000] != executed_actions[:2000]) & (executed_actions[:2000] == 1)
invalid_signals = timestamps[:2000][invalid_mask]
invalid_prices = price_data[:2000][invalid_mask]
fig.add_trace(
    go.Scatter(x=invalid_signals, y=invalid_prices, mode='markers', name="Invalid Actions",
               marker=dict(color="orange", symbol="x", size=6)),
    row=1, col=1
)

# Panel 2: Portfolio value
fig.add_trace(
    go.Scatter(x=timestamps[:2000], y=portfolio_values[:2000], name="Portfolio Value", 
               line=dict(color="purple", width=2)),
    row=2, col=1
)

# Panel 3: Message frequency
time_diffs = np.diff(timestamps[:2000])
fig.add_trace(
    go.Scatter(x=timestamps[:1999], y=time_diffs, name="Time Between Messages", 
               line=dict(color="orange", width=1)),
    row=3, col=1
)

# Update layout
fig.update_layout(
    height=1000,
    width=1200,
    title_text="Time-Aware Neural Network Trading Analysis (Position-Aware)",
    showlegend=True
)

# Update y-axis labels
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Portfolio Value", row=2, col=1)
fig.update_yaxes(title_text="Time Diff", row=3, col=1)
fig.update_xaxes(title_text="Time", row=3, col=1)

fig.show()

## Temporal Memory Analysis
Compare network behavior during rush vs normal periods

In [16]:
def analyze_rush_periods(prices, timestamps, executed_actions):
    time_diffs = np.diff(timestamps)
    rush_mask = time_diffs[:-1] < 0.5  # Rush periods have small time increments
    normal_mask = time_diffs[:-1] >= 0.5
    
    rush_actions = executed_actions[1:-1][rush_mask]
    normal_actions = executed_actions[1:-1][normal_mask]
    
    return rush_actions, normal_actions

rush_actions, normal_actions = analyze_rush_periods(price_data, timestamps, executed_actions)

print("Rush Period Action Distribution (Executed):")
print(f"  Hold: {np.sum(rush_actions==1)/len(rush_actions)*100:.1f}%")
print(f"  Sell: {np.sum(rush_actions==0)/len(rush_actions)*100:.1f}%")
print(f"  Buy:  {np.sum(rush_actions==2)/len(rush_actions)*100:.1f}%")

print("\nNormal Period Action Distribution (Executed):")
print(f"  Hold: {np.sum(normal_actions==1)/len(normal_actions)*100:.1f}%")
print(f"  Sell: {np.sum(normal_actions==0)/len(normal_actions)*100:.1f}%")
print(f"  Buy:  {np.sum(normal_actions==2)/len(normal_actions)*100:.1f}%")

Rush Period Action Distribution (Executed):
  Hold: 98.2%
  Sell: 0.8%
  Buy:  1.0%

Normal Period Action Distribution (Executed):
  Hold: 100.0%
  Sell: 0.0%
  Buy:  0.0%


In [17]:
# Multiple individuals performance comparison
n_individuals = 20
results = []

for i in range(n_individuals):
    individual = Individual()
    _, _, portfolio_values = simulate_trading(individual, price_data, timestamps)
    final_value = portfolio_values[-1]
    results.append(final_value)

results = np.array(results)
print(f"Performance Statistics ({n_individuals} individuals):")
print(f"  Mean: {np.mean(results):.4f}")
print(f"  Std:  {np.std(results):.4f}")
print(f"  Best: {np.max(results):.4f}")
print(f"  Worst: {np.min(results):.4f}")

# Create interactive histogram with plotly
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=results,
    nbinsx=15,
    name="Portfolio Performance",
    marker_color="skyblue",
    marker_line_color="black",
    marker_line_width=1
))

# Add mean line
fig.add_vline(
    x=np.mean(results), 
    line_dash="dash", 
    line_color="red",
    annotation_text=f"Mean: {np.mean(results):.4f}",
    annotation_position="top"
)

fig.update_layout(
    title="Portfolio Performance Distribution (Position-Aware Trading)",
    xaxis_title="Final Portfolio Value",
    yaxis_title="Frequency",
    width=800,
    height=500,
    showlegend=False
)

fig.show()

Performance Statistics (20 individuals):
  Mean: 1.0972
  Std:  0.2560
  Best: 2.0420
  Worst: 0.8423
